In [ ]:
%matplotlib ipympl
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import pandas as pd
import keras
print(tf. __version__)
print(keras.__version__)

In [ ]:
train_url = 'https://raw.githubusercontent.com/Tim0n1/MLDL_zapiski/master/train.csv'
test_url_X = 'https://raw.githubusercontent.com/Tim0n1/MLDL_zapiski/master/test.csv'
test_url_y = 'https://raw.githubusercontent.com/Tim0n1/MLDL_zapiski/master/gender_submission.csv'

In [ ]:
train_df = pd.read_csv(train_url)
testX_df = pd.read_csv(test_url_X)
testy_df = pd.read_csv(test_url_y)

In [ ]:

train_X = train_df.drop(columns=['Survived','PassengerId','Name','Ticket', 'Embarked','Fare'])
train_X.loc[train_X['Sex'] == 'male' , ['Sex']] = 1
train_X.loc[train_X['Sex'] == 'female' , ['Sex']] = 0
train_X = train_X.fillna(0)
train_X.loc[train_X['Cabin'] != 0, ['Cabin']] = 1
print(train_X.shape)
train_y = train_df['Survived']
print(train_y.shape)
test_X = testX_df.drop(columns=['PassengerId','Name','Ticket', 'Embarked','Fare'])
test_X.loc[test_X['Sex'] == 'male' , ['Sex']] = 1
test_X.loc[test_X['Sex'] == 'female' , ['Sex']] = 0
test_X = test_X.fillna(0)
test_X.loc[test_X['Cabin'] != 0, ['Cabin']] = 1
print(test_X)
test_y = testy_df['Survived']

In [ ]:
input_shape = train_X.shape

In [ ]:
train_X.to_numpy()
train_y.to_numpy()
test_X.to_numpy()
test_y.to_numpy()

In [ ]:
train_X = np.asarray(train_X).astype(np.float32)
train_y = np.asarray(train_y).astype(np.float32)
test_X = np.asarray(test_X).astype(np.float32)
test_y = np.asarray(test_y).astype(np.float32)

In [ ]:
print((train_X))
print(len(train_y))

In [ ]:
train_X = train_X.reshape(891,6)
train_y = train_y.reshape(891)
test_X = test_X.reshape(418,6)
test_y = test_y.reshape(418)

In [ ]:
train_X.shape, train_y.shape

In [ ]:
test_X.shape, test_y.shape

In [ ]:
model = keras.Sequential()
model.add(keras.layers.Dense(16, input_dim=train_X.shape[1], activation='relu'))
model.add(keras.layers.Dense(8, activation='relu'))
model.add(keras.layers.Dense(4, activation='relu'))
model.add(keras.layers.Dense(1, activation='sigmoid'))

In [ ]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
model.fit(train_X, train_y, epochs=50,batch_size=6, validation_data=(test_X, test_y), verbose=1)

In [ ]:
model.save('model_data')

In [ ]:
model = keras.models.load_model('model_data')

In [ ]:

def get_predicted_survivors(x):
    predicted_survivors = []
    model_prediction = model.predict(test_X[:])
    for n, i in enumerate(model_prediction):
        if (i[0] >= x-0.1) and (i[0] <= x+0.1):
            predicted_survivors.append(n)
            #print(n,i[0])
    return predicted_survivors

## Do some statistic about the prediction accuracy of the model

In [ ]:
print(get_predicted_survivors(0.555))


In [ ]:
y = []
x = np.linspace(0,1,num=100)
n_1_l = []
for i in x:
    n_1 = 0
    predicted_val = test_y[get_predicted_survivors(i)]
    for i in predicted_val:
        if i == 1:
            n_1 += 1
    print(n_1)
    accuracy = predicted_val.mean()
    n_1_l.append(n_1)
    y.append(accuracy)
print(y)


In [ ]:
#%matplotlib ipympl
%matplotlib inline
fig, ax = plt.subplots(2,1)
ax[0].plot(x, y, label='Accuracy')
#plt.plot(x, n_1_l)
#plt.plot(x, np.gradient(n_1_l))
ax[0].plot(x, np.gradient(y), label='Deriviative of Accuracy')
ax[0].set_xlabel('Certainty of the model')
ax[0].set_ylabel('Right predicts/All predicts')
ax[0].legend()
ax[1].plot(x, n_1_l, label='N')
ax[1].plot(x, np.gradient(n_1_l), label='Deriviative of N')
ax[1].set_xlabel('Certainty of the model')
ax[1].set_ylabel('N right predicts')
ax[1].legend()
plt.style.use('ggplot')
fig.tight_layout()

In [ ]:
fig.savefig('prediction_analysis.png')

## Inserting with custom data

In [ ]:
test_input = [3,1,40,0,0,1]
def get_potential_survive_chance(input):
    input = np.array(input)
    input.resize(1,6)
    prediction = model.predict(input)
    print(prediction)
    prediction = prediction[0,0]
    chance = test_y[get_predicted_survivors(prediction)].mean()
    if chance == np.nan:
        chance = prediction
        print(prediction)
    print(get_predicted_survivors(prediction))
    print(test_y[get_predicted_survivors(prediction)])
    print(chance)
    return np.round(chance*100, 2)
print(get_potential_survive_chance(test_input))

In [ ]:
#todo the function needs more research. plot the different predict settings to find best solution